# CrystalMath Quick Start Guide

This notebook demonstrates the basic usage of CrystalMath for materials analysis.
You will learn how to:

1. Import the CrystalMath module
2. Load a crystal structure from a CIF file
3. Run a basic SCF calculation
4. Access and interpret results
5. Export results to CSV and generate plots

**Prerequisites:**
- CrystalMath installed (`pip install crystalmath[all]`)
- A CIF file to analyze (we'll use a provided example)

## 1. Import CrystalMath

The main entry point is the `HighThroughput` class from `crystalmath.high_level`.

In [ ]:
# Import the high-level API
from crystalmath.high_level import HighThroughput

# Optional: Import pymatgen for structure manipulation
from pymatgen.core import Structure

# For data handling and visualization
import pandas as pd
import matplotlib.pyplot as plt

print("CrystalMath imported successfully!")

## 2. Load a Structure from CIF

CrystalMath supports multiple structure formats:
- CIF files (`.cif`)
- VASP POSCAR files
- XYZ files
- Materials Project IDs (`mp-149`)
- pymatgen Structure objects

In [ ]:
# Load a structure from a CIF file
# Replace 'Si.cif' with the path to your structure file
structure_path = "Si.cif"

# You can also load directly from pymatgen
structure = Structure.from_file(structure_path)

# Print structure information
print(f"Formula: {structure.composition.reduced_formula}")
print(f"Number of atoms: {len(structure)}")
print(f"Lattice parameters: a={structure.lattice.a:.3f}, b={structure.lattice.b:.3f}, c={structure.lattice.c:.3f}")
print(f"Volume: {structure.volume:.3f} A^3")

## 3. Run a Basic SCF Calculation

The `run_standard_analysis()` method provides a one-liner interface for complete workflows.

**Protocol levels:**
- `fast`: Quick screening (coarse k-mesh)
- `moderate`: Production quality (default)
- `precise`: Publication quality (fine k-mesh)

In [ ]:
# Run a basic SCF calculation
results = HighThroughput.run_standard_analysis(
    structure=structure_path,
    properties=["scf"],        # Just single-point energy
    protocol="moderate",       # Production quality
    codes={"dft": "vasp"},     # Use VASP (or "crystal23", "quantum_espresso")
    cluster="local"            # Run locally (or specify cluster name)
)

print("SCF calculation completed!")

### Adding More Properties

You can request multiple properties in a single workflow:

In [ ]:
# Run a more complete analysis
results = HighThroughput.run_standard_analysis(
    structure=structure_path,
    properties=["relax", "bands", "dos"],  # Relaxation + electronic structure
    protocol="moderate",
    codes={"dft": "vasp"},
    cluster="beefcake2"  # Use the HPC cluster
)

print("Analysis completed!")

## 4. Access Results

The `AnalysisResults` object provides easy access to all computed properties.

In [ ]:
# Access scalar properties
print(f"Formula: {results.formula}")
print(f"Space group: {results.space_group}")
print(f"Band gap: {results.band_gap_ev:.3f} eV")
print(f"Direct gap: {results.is_direct_gap}")
print(f"Is metal: {results.is_metal}")
print(f"Fermi energy: {results.fermi_energy_ev:.3f} eV")

In [ ]:
# Check what properties are available
supported = HighThroughput.get_supported_properties()
print("Supported properties:")
for prop in supported:
    info = HighThroughput.get_property_info(prop)
    print(f"  - {prop}: default code = {info['default_code']}, dependencies = {info['dependencies']}")

## 5. Export Results

CrystalMath provides multiple export formats for results.

In [ ]:
# Export to pandas DataFrame
df = results.to_dataframe()
print("DataFrame columns:")
print(df.columns.tolist())
df

In [ ]:
# Save to CSV
df.to_csv("silicon_results.csv", index=False)
print("Results saved to silicon_results.csv")

In [ ]:
# Export to JSON
results.to_json("silicon_results.json")
print("Results saved to silicon_results.json")

# Or get JSON string
json_str = results.to_json()
print(json_str[:500] + "...")

## 6. Generate Plots

CrystalMath includes built-in plotting methods for publication-quality figures.

In [ ]:
# Plot band structure
fig = results.plot_bands()
fig.savefig("silicon_bands.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Plot density of states
fig = results.plot_dos(projected=True)
fig.savefig("silicon_dos.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Combined band structure and DOS plot (publication format)
fig = results.plot_bands_dos(figsize=(10, 6))
fig.savefig("silicon_electronic_structure.png", dpi=300, bbox_inches='tight')
plt.show()

## 7. LaTeX Export for Publications

Generate publication-ready LaTeX tables.

In [ ]:
# Generate LaTeX table
latex_table = results.to_latex_table()
print(latex_table)

In [ ]:
# Save to file
results.to_latex_table("silicon_table.tex")
print("LaTeX table saved to silicon_table.tex")

## Summary

In this notebook, you learned how to:

1. **Import** CrystalMath and related libraries
2. **Load** structures from CIF files
3. **Run** SCF and electronic structure calculations
4. **Access** computed properties (band gap, Fermi energy, etc.)
5. **Export** results to CSV, JSON, and LaTeX formats
6. **Generate** publication-quality plots

**Next steps:**
- Try `02_materials_project.ipynb` to learn about fetching structures from Materials Project
- See `03_band_structure.ipynb` for detailed band structure analysis
- Explore `04_optical_properties.ipynb` for GW/BSE calculations